In [1]:


import torch
from train import MCTS_HYPERPARAMETERS, ReplayMemory, MetricsHistory, load_from_checkpoint, test_network, collect_episode, train, rotate_training_examples
from env import _2048Env
from model_3d import MonteCarlo3d
from mcts import MCTS_Evaluator
from utils import input_to_tensor_3d

In [2]:
MODEL_TO_LOAD = "" # place filename of checkpoint here, otherwise leave empty
LOAD_REPLAY_MEMORY = True

# HYPERPARAMETERS (will be ignored if loading from a checkpoint)
hyperparameters = MCTS_HYPERPARAMETERS() # use kwargs to specify non-default values

In [3]:
# if MODEL_TO_LOAD:
#     env, mcts, episode, model, optimizer, hyperparameters, metric_history, replay_memory, run_tag = load_from_checkpoint(MODEL_TO_LOAD, load_replay_memory=LOAD_REPLAY_MEMORY)
# else:
#     env = _2048Env()
#     model = MonteCarlo3d()
#     mcts = MCTS_Evaluator(model, env, input_to_tensor_3d, training=True)
#     replay_memory = ReplayMemory(hyperparameters.replay_memory_size)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparameters.lr, weight_decay=hyperparameters.weight_decay)
#     metric_history = MetricsHistory()
#     run_tag = ''     
#     episode = 0



In [4]:
# hyperparameters.num_mcts_test_evals = 500
# test_network(model, hyperparameters, input_to_tensor_3d)

In [6]:
import torch.multiprocessing as mp

replay_memory = ReplayMemory(hyperparameters.replay_memory_size)
metrics_history = MetricsHistory()
model = MonteCarlo3d()
optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparameters.lr, weight_decay=hyperparameters.weight_decay)
model.share_memory()
hyperparameters.minibatch_size = 4000
hyperparameters.replay_memory_size = 120000
def enque_and_train(results):
    training_examples, reward, moves, high_square= results
    training_examples = rotate_training_examples(training_examples)
    replay_memory.extend(training_examples)
    
    if replay_memory.size() >= hyperparameters.minibatch_size:
        value_loss, prob_loss, total_loss = train(replay_memory.sample(hyperparameters.minibatch_size), model, optimizer, tensor_conversion_fn=input_to_tensor_3d)
        new_best = metrics_history.add_history({
                'reward': reward,
                'game_moves': moves,
                'prob_loss': prob_loss,
                'value_loss': value_loss,
                'total_loss': total_loss,
                'high_square': high_square
            })
        metrics_history.plot_history()
        if new_best:
            print('*** NEW BEST REWARD ***')
        print(f'[EPISODE {metrics_history.episodes}] Total Loss: {total_loss}, Prob Loss {prob_loss}, Value Loss {value_loss}, Reward {reward}, Moves: {moves}, Highest Square: {high_square}')
    else:
        print(f'Replay memory size not large enough, {replay_memory.size()} < {hyperparameters.minibatch_size}')
    
    

with mp.Pool(mp.cpu_count() - 1) as p:
    results = []
    for n in range(hyperparameters.num_episodes):
        results.append(p.apply_async(collect_episode, (model, hyperparameters, input_to_tensor_3d, ), callback=enque_and_train, error_callback=print))
    for r in results:
        r.wait()

Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/Users/marshingjay/opt/anaconda3/envs/python38/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2723, in settrace
  File "/Users/marshingjay/opt/anaconda3/envs/python38/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2723, in settrace
  File "/Users/marshingjay/opt/anaconda3/envs/python38/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2723, in settrace
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/marshingjay/opt/anaconda3/envs/python38/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2723, in settrace
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/marshingjay/opt/anaconda3/envs/python38/lib

KeyboardInterrupt: 